# Tiền xử lý dữ liệu Airbnb NYC 2019

## 1. Import thư viện và load dữ liệu

In [1]:
import sys
import os
import numpy as np

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))
from data_processing import *

In [2]:
file_path = '../data/raw/AB_NYC_2019.csv'
data_numpy = read_csv(file_path)

column_names = data_numpy[0]
data = data_numpy[1:]

print("Dữ liệu đã được load thành công!")
print(f"Shape ban đầu: {data.shape}")
print(f"Số cột ban đầu: {len(column_names)}")

Dữ liệu đã được load thành công!
Shape ban đầu: (48895, 16)
Số cột ban đầu: 16


## 2. Kiểm tra dữ liệu ban đầu

In [3]:
print("Danh sách các cột ban đầu:")
for i, col in enumerate(column_names):
    print(f"  {i+1}. {col}")
    
print(f"\n{'='*50}")
print("Missing values trong dữ liệu ban đầu:")
for i, col in enumerate(column_names):
    col_data = data[:, i]
    missing_count = np.sum(col_data == '')
    
    if missing_count > 0:
        missing_percent = (missing_count / len(data) * 100)
        print(f"\n{col}:")
        print(f"  - Số lượng: {missing_count:,}")
        print(f"  - Tỷ lệ: {missing_percent:.2f}%")

Danh sách các cột ban đầu:
  1. id
  2. name
  3. host_id
  4. host_name
  5. neighbourhood_group
  6. neighbourhood
  7. latitude
  8. longitude
  9. room_type
  10. price
  11. minimum_nights
  12. number_of_reviews
  13. last_review
  14. reviews_per_month
  15. calculated_host_listings_count
  16. availability_365

Missing values trong dữ liệu ban đầu:

name:
  - Số lượng: 16
  - Tỷ lệ: 0.03%

host_name:
  - Số lượng: 21
  - Tỷ lệ: 0.04%

last_review:
  - Số lượng: 10,052
  - Tỷ lệ: 20.56%

reviews_per_month:
  - Số lượng: 10,052
  - Tỷ lệ: 20.56%


## 3. Loại bỏ các cột không cần thiết

**Các cột cần drop:**
- `id`: Không có giá trị sử dụng cho mục đích tính toán/phân tích
- `name`: Chỉ là tên mô tả, không có giá trị số liệu
- `host_name`: 
  - Có 21 missing values
  - Đã có `host_id` thay thế với dữ liệu đầy đủ hơn
- `last_review`: 
  - Không quan trọng bằng số lượng review
  - Đã có `number_of_reviews` và `reviews_per_month` để phân tích

In [4]:
columns_to_drop = ['id', 'name', 'host_name', 'last_review']

is_drop_col = np.isin(column_names, columns_to_drop)

keep_mask = ~is_drop_col

data_cleaned = data[:, keep_mask]
column_names_cleaned = column_names[keep_mask]

print(f"Shape trước khi drop: {data.shape}")
print(f"Shape sau khi drop: {data_cleaned.shape}")
print(f"\nCác cột đã loại bỏ: {columns_to_drop}")

print(f"\nCác cột còn lại ({len(column_names_cleaned)}):")
print("\n".join([f"{i+1}. {col}" for i, col in enumerate(column_names_cleaned)]))


Shape trước khi drop: (48895, 16)
Shape sau khi drop: (48895, 12)

Các cột đã loại bỏ: ['id', 'name', 'host_name', 'last_review']

Các cột còn lại (12):
1. host_id
2. neighbourhood_group
3. neighbourhood
4. latitude
5. longitude
6. room_type
7. price
8. minimum_nights
9. number_of_reviews
10. reviews_per_month
11. calculated_host_listings_count
12. availability_365


## 4. Xử lý missing values trong reviews_per_month

**Lý do gán giá trị 0.00:**
- Cột `reviews_per_month` có 10,052 missing values
- Khi đối chiếu với `number_of_reviews`, những listing có `reviews_per_month` bị thiếu đều có `number_of_reviews = 0`
- Logic: Nếu không có review nào thì trung bình reviews/tháng = 0.00

In [5]:
reviews_per_month_idx = np.where(column_names_cleaned == 'reviews_per_month')[0][0]

mask_missing = (data_cleaned[:, reviews_per_month_idx] == '')

missing_before = np.sum(mask_missing)
print(f"Missing values trước khi xử lý: {missing_before:,}")

data_cleaned[mask_missing, reviews_per_month_idx] = '0.00'
missing_after = np.sum(data_cleaned[:, reviews_per_month_idx] == '')

print(f"Missing values sau khi xử lý: {missing_after:,}")
print(f"\nĐã thay thế {missing_before:,} missing values bằng 0.00")

Missing values trước khi xử lý: 10,052
Missing values sau khi xử lý: 0

Đã thay thế 10,052 missing values bằng 0.00


## 5. Chuyển đổi dữ liệu sang kiểu phù hợp

**Mục đích:** Đảm bảo kiểu dữ liệu nhất quán và tối ưu cho tính toán
- String columns: `astype(str)` 
- Numeric columns: `astype(np.float64)` cho độ chính xác cao

**Kỹ thuật:** Sử dụng `pandas.DataFrame.astype()` để chuyển đổi kiểu dữ liệu của các cột một cách hiệu quả và nhanh chóng.

In [6]:
neighbourhood_group_idx = np.where(column_names_cleaned == 'neighbourhood_group')[0][0]
neighbourhood_idx = np.where(column_names_cleaned == 'neighbourhood')[0][0]
room_type_idx = np.where(column_names_cleaned == 'room_type')[0][0]
latitude_idx = np.where(column_names_cleaned == 'latitude')[0][0]
longitude_idx = np.where(column_names_cleaned == 'longitude')[0][0]
price_idx = np.where(column_names_cleaned == 'price')[0][0]
min_nights_idx = np.where(column_names_cleaned == 'minimum_nights')[0][0]
num_reviews_idx = np.where(column_names_cleaned == 'number_of_reviews')[0][0]
reviews_per_month_idx = np.where(column_names_cleaned == 'reviews_per_month')[0][0]
calc_host_idx = np.where(column_names_cleaned == 'calculated_host_listings_count')[0][0]
availability_idx = np.where(column_names_cleaned == 'availability_365')[0][0]

neighbourhood_groups = data_cleaned[:, neighbourhood_group_idx]
neighbourhoods = data_cleaned[:, neighbourhood_idx]
room_types = data_cleaned[:, room_type_idx]

latitudes = data_cleaned[:, latitude_idx].astype(float)
longitudes = data_cleaned[:, longitude_idx].astype(float)
prices = data_cleaned[:, price_idx].astype(float)
min_nights = data_cleaned[:, min_nights_idx].astype(float)
number_of_reviews = data_cleaned[:, num_reviews_idx].astype(float)
calc_host_listings = data_cleaned[:, calc_host_idx].astype(float)
reviews_per_month = data_cleaned[:, reviews_per_month_idx].astype(float)
availability = data_cleaned[:, availability_idx].astype(float)

print("\nDữ liệu đã được xử lý xong!")


Dữ liệu đã được xử lý xong!



## 6. Lọc dữ liệu nhiễu

**Mục đích:** Loại bỏ các bản ghi có giá trị không hợp lệ để đảm bảo chất lượng dữ liệu
- Tập trung vào cột `price`

**Kỹ thuật:** Sử dụng Boolean Masking - Fancy Indexing để lọc dữ liệu không hợp lệ (price < 0)
- Không cần loop
- Memory efficient (views thay vì copies)
- Syntax sạch và dễ đọc

In [7]:
valid_mask = prices > 0 
print(f"Số dòng trước khi lọc: {len(prices):,}")
print(f"Số dòng có price <= 0: {np.sum(~valid_mask):,}")
print(f"Số dòng sau khi lọc: {np.sum(valid_mask):,}")

prices = prices[valid_mask]
min_nights = min_nights[valid_mask]
latitudes = latitudes[valid_mask]
longitudes = longitudes[valid_mask]
neighbourhood_groups = neighbourhood_groups[valid_mask]
neighbourhoods = neighbourhoods[valid_mask]
room_types = room_types[valid_mask]
number_of_reviews = number_of_reviews[valid_mask]
reviews_per_month = reviews_per_month[valid_mask]
calc_host_listings = calc_host_listings[valid_mask]
availability = availability[valid_mask]

print(f"\nĐã lọc thành công, còn lại {len(prices):,} dòng dữ liệu hợp lệ")

Số dòng trước khi lọc: 48,895
Số dòng có price <= 0: 11
Số dòng sau khi lọc: 48,884

Đã lọc thành công, còn lại 48,884 dòng dữ liệu hợp lệ

Đã lọc thành công, còn lại 48,884 dòng dữ liệu hợp lệ


## 7. One-hot encoding cho cột neighbourhood_group

**Mục đích:** Chuyển đổi biến phân loại `neighbourhood_group` và `room_type` thành dạng số để sử dụng trong mô hình học máy
- Tạo các cột nhị phân riêng biệt cho mỗi giá trị trong `neighbourhood_group` và `room_type`

**Kỹ thuật:** One-hot encoding sử dụng broadcasting để chuyển đổi cột `neighbourhood_group` và `room_type` thành các cột nhị phân
- Broadcasting: `(N,1) == (C,)` → `(N,C)` 
- Tốc độ: O(N×C) nhưng vectorized nên nhanh hơn nhiều so với loop

In [8]:
def fast_one_hot(arr):
    """
    One-hot encoding dùng Broadcasting
    arr: (N,) -> output: (N, C) với C là số classes unique
    """
    classes = np.unique(arr)
    # Broadcasting: arr[:, None] shape (N,1), classes shape (C,)
    # Kết quả so sánh: (N,1) == (C,) -> (N,C)
    return (arr[:, None] == classes).astype(int)

# Áp dụng one-hot encoding
oh_neighbourhood_group = fast_one_hot(neighbourhood_groups)
oh_room_type = fast_one_hot(room_types)

print("One-Hot Encoding Results:")
print(f"- neighbourhood_group: {oh_neighbourhood_group.shape}")
print(f"  Classes: {np.unique(neighbourhood_groups)}")
print(f"\n- room_type: {oh_room_type.shape}")
print(f"  Classes: {np.unique(room_types)}")

# Hiển thị mẫu
print(f"\nMẫu one-hot encoding cho room_type (5 dòng đầu):")
print(oh_room_type[:5])

One-Hot Encoding Results:
- neighbourhood_group: (48884, 5)
  Classes: ['Bronx' 'Brooklyn' 'Manhattan' 'Queens' 'Staten Island']

- room_type: (48884, 3)
  Classes: ['Entire home/apt' 'Private room' 'Shared room']

Mẫu one-hot encoding cho room_type (5 dòng đầu):
[[0 1 0]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]]


## 7 Chia tập train và test

**Mục đích:** Chia dữ liệu thành tập train (80%) và test (20%) trước khi thực hiện Feature Engineering
- Tránh data leakage khi áp dụng target encoding và scaling
- Đảm bảo test set độc lập hoàn toàn với train set

**Kỹ thuật:** Random sampling với np.random.permutation
- Shuffle dữ liệu ngẫu nhiên
- Chia theo tỷ lệ 80-20

In [9]:
# Set random seed để đảm bảo reproducibility
np.random.seed(42)

# Tổng số samples
n_samples = len(prices)

# Tạo indices ngẫu nhiên
shuffled_indices = np.random.permutation(n_samples)

# Tính số lượng cho train (80%)
train_size = int(0.8 * n_samples)

# Chia indices
train_indices = shuffled_indices[:train_size]
test_indices = shuffled_indices[train_size:]

# Chia các arrays thành train và test
# Numerical features
prices_train, prices_test = prices[train_indices], prices[test_indices]
latitudes_train, latitudes_test = latitudes[train_indices], latitudes[test_indices]
longitudes_train, longitudes_test = longitudes[train_indices], longitudes[test_indices]
min_nights_train, min_nights_test = min_nights[train_indices], min_nights[test_indices]
number_of_reviews_train, number_of_reviews_test = number_of_reviews[train_indices], number_of_reviews[test_indices]
reviews_per_month_train, reviews_per_month_test = reviews_per_month[train_indices], reviews_per_month[test_indices]
calc_host_listings_train, calc_host_listings_test = calc_host_listings[train_indices], calc_host_listings[test_indices]
availability_train, availability_test = availability[train_indices], availability[test_indices]

# Categorical features
neighbourhood_groups_train, neighbourhood_groups_test = neighbourhood_groups[train_indices], neighbourhood_groups[test_indices]
neighbourhoods_train, neighbourhoods_test = neighbourhoods[train_indices], neighbourhoods[test_indices]
room_types_train, room_types_test = room_types[train_indices], room_types[test_indices]

# One-hot encoded features (đã tạo ở bước 7)
oh_neighbourhood_group_train, oh_neighbourhood_group_test = oh_neighbourhood_group[train_indices], oh_neighbourhood_group[test_indices]
oh_room_type_train, oh_room_type_test = oh_room_type[train_indices], oh_room_type[test_indices]

print("Chia dữ liệu thành train và test:")
print(f"- Tổng số samples: {n_samples:,}")
print(f"- Train set: {len(train_indices):,} samples ({len(train_indices)/n_samples*100:.1f}%)")
print(f"- Test set: {len(test_indices):,} samples ({len(test_indices)/n_samples*100:.1f}%)")
print(f"\nKiểm tra shape:")
print(f"- prices_train: {prices_train.shape}")
print(f"- prices_test: {prices_test.shape}")
print(f"- neighbourhoods_train: {neighbourhoods_train.shape}")
print(f"- neighbourhoods_test: {neighbourhoods_test.shape}")

Chia dữ liệu thành train và test:
- Tổng số samples: 48,884
- Train set: 39,107 samples (80.0%)
- Test set: 9,777 samples (20.0%)

Kiểm tra shape:
- prices_train: (39107,)
- prices_test: (9777,)
- neighbourhoods_train: (39107,)
- neighbourhoods_test: (9777,)


## 8. Feature Engineering - Tạo features mới

**Mục đích:** Tạo các biến mới từ dữ liệu gốc để cải thiện hiệu suất mô hình học máy

### 8.1. Tính khoảng cách tới Times Square

**Mục đích:** Tính khoảng cách từ mỗi listing tới Times Square để sử dụng làm feature trong mô hình học máy
- Times Square tọa độ: (40.7580, -73.9855)

**Kỹ thuật:** Sử dụng `np.einsum` cho đại số tuyến tính
- Tính khoảng cách từ mỗi listing tới Times Square (40.7580, -73.9855)
- `einsum('ij,ij->i')`: Nhân element-wise rồi sum theo axis 1
- Hiệu quả hơn nhiều so với loop hoặc `np.sum()`

In [10]:
# Times Square coordinates
center = np.array([40.7580, -73.9855])

# Tính khoảng cách cho train set
coords_train = np.column_stack((latitudes_train, longitudes_train))
diff_train = coords_train - center
dist_sq_train = np.einsum('ij,ij->i', diff_train, diff_train)
dist_to_center_train = np.sqrt(dist_sq_train)

# Tính khoảng cách cho test set
coords_test = np.column_stack((latitudes_test, longitudes_test))
diff_test = coords_test - center
dist_sq_test = np.einsum('ij,ij->i', diff_test, diff_test)
dist_to_center_test = np.sqrt(dist_sq_test)

print("Khoảng cách tới Times Square:")
print(f"\nTrain set:")
print(f"  - Min: {dist_to_center_train.min():.4f} degrees")
print(f"  - Max: {dist_to_center_train.max():.4f} degrees")
print(f"  - Mean: {dist_to_center_train.mean():.4f} degrees")
print(f"\nTest set:")
print(f"  - Min: {dist_to_center_test.min():.4f} degrees")
print(f"  - Max: {dist_to_center_test.max():.4f} degrees")
print(f"  - Mean: {dist_to_center_test.mean():.4f} degrees")

Khoảng cách tới Times Square:

Train set:
  - Min: 0.0007 degrees
  - Max: 0.3594 degrees
  - Mean: 0.0706 degrees

Test set:
  - Min: 0.0007 degrees
  - Max: 0.3631 degrees
  - Mean: 0.0699 degrees


### 8.2 Binning cho minimum_nights

**Mục đích:** 
- Loại bỏ ảnh hưởng của các giá trị ngoại lai (outliers) cực đoan (ví dụ: 1250 đêm) vốn làm sai lệch các thuật toán tính toán khoảng cách.
- Phân loại minimum_nights thành các nhóm có ý nghĩa
    - Bin 0: ≤3 đêm (du khách ngắn hạn)
    - Bin 1: 4-7 đêm (thuê theo tuần)
    - Bin 2: 8-31 đêm (thuê theo tháng)
    - Bin 3: >31 đêm (dài hạn)

**Kỹ thuật:** 
- `np.digitize(arr, bins, right=True)`: Phân loại giá trị vào bins
- `np.eye(num_categories)[indices]`: Tạo one-hot encoding từ indices

In [11]:
# Định nghĩa ranh giới bins
bins = np.array([3, 7, 31])

# Binning cho train set
min_nights_binned_indices_train = np.digitize(min_nights_train, bins, right=True)
num_categories = 4
oh_min_nights_train = np.eye(num_categories)[min_nights_binned_indices_train]

# Binning cho test set
min_nights_binned_indices_test = np.digitize(min_nights_test, bins, right=True)
oh_min_nights_test = np.eye(num_categories)[min_nights_binned_indices_test]

print("Binning minimum_nights:")
print(f"\nTrain set - oh_min_nights shape: {oh_min_nights_train.shape}")
print("Phân phối:")
for i in range(num_categories):
    count = np.sum(min_nights_binned_indices_train == i)
    percentage = (count / len(min_nights_binned_indices_train)) * 100
    if i == 0:
        label = "≤3 nights (short-term)"
    elif i == 1:
        label = "4-7 nights (weekly)"
    elif i == 2:
        label = "8-31 nights (monthly)"
    else:
        label = ">31 nights (long-term)"
    print(f"  Bin {i} ({label}): {count} listings ({percentage:.2f}%)")

print(f"\nTest set - oh_min_nights shape: {oh_min_nights_test.shape}")
print("Phân phối:")
for i in range(num_categories):
    count = np.sum(min_nights_binned_indices_test == i)
    percentage = (count / len(min_nights_binned_indices_test)) * 100
    if i == 0:
        label = "≤3 nights (short-term)"
    elif i == 1:
        label = "4-7 nights (weekly)"
    elif i == 2:
        label = "8-31 nights (monthly)"
    else:
        label = ">31 nights (long-term)"
    print(f"  Bin {i} ({label}): {count} listings ({percentage:.2f}%)")

Binning minimum_nights:

Train set - oh_min_nights shape: (39107, 4)
Phân phối:
  Bin 0 (≤3 nights (short-term)): 25960 listings (66.38%)
  Bin 1 (4-7 nights (weekly)): 7334 listings (18.75%)
  Bin 2 (8-31 nights (monthly)): 5374 listings (13.74%)
  Bin 3 (>31 nights (long-term)): 439 listings (1.12%)

Test set - oh_min_nights shape: (9777, 4)
Phân phối:
  Bin 0 (≤3 nights (short-term)): 6448 listings (65.95%)
  Bin 1 (4-7 nights (weekly)): 1811 listings (18.52%)
  Bin 2 (8-31 nights (monthly)): 1411 listings (14.43%)
  Bin 3 (>31 nights (long-term)): 107 listings (1.09%)


### 8.3 Tính log-transform của calc_host_listings_log

**Mục đích:**
- Xử lý vấn đề phân phối "đuôi dài" (Long-tail distribution): Đại đa số host chỉ có 1 nhà hoặc ít listing, nhưng một số ít lại có rất nhiều (lên đến hàng trăm).
- Thu hẹp khoảng cách giá trị để mô hình không bị "đè bẹp" bởi các con số quá lớn, đồng thời vẫn giữ được thứ tự xếp hạng quan trọng.

**Kỹ thuật:**
- Log Transformation (log1p): Sử dụng hàm $y = \ln(x + 1)$.
- `log1p` tránh vấn đề log(0) và cung cấp độ chính xác cao hơn cho các giá trị nhỏ

In [12]:
# Log-transform cho train set
calc_host_listings_log_train = np.log1p(calc_host_listings_train)

# Log-transform cho test set
calc_host_listings_log_test = np.log1p(calc_host_listings_test)

print("Log-Transform Results for calc_host_listings:")
print(f"\nTrain set:")
print(f"  - Original range: [{calc_host_listings_train.min()}, {calc_host_listings_train.max()}]")
print(f"  - Log-Transformed range: [{calc_host_listings_log_train.min():.4f}, {calc_host_listings_log_train.max():.4f}]")
print(f"\nTest set:")
print(f"  - Original range: [{calc_host_listings_test.min()}, {calc_host_listings_test.max()}]")
print(f"  - Log-Transformed range: [{calc_host_listings_log_test.min():.4f}, {calc_host_listings_log_test.max():.4f}]")

Log-Transform Results for calc_host_listings:

Train set:
  - Original range: [1.0, 327.0]
  - Log-Transformed range: [0.6931, 5.7930]

Test set:
  - Original range: [1.0, 327.0]
  - Log-Transformed range: [0.6931, 5.7930]


### 8.4 Tính log-transform của number_of_reviews_log

**Mục đích:**
- Xử lý vấn đề phân phối "đuôi dài" (Long-tail distribution): Đại đa số host chỉ có 1 nhà hoặc ít review, nhưng một số ít lại có rất nhiều (lên đến hàng trăm).
- Thu hẹp khoảng cách giá trị để mô hình không bị "đè bẹp" bởi các con số quá lớn, đồng thời vẫn giữ được thứ tự xếp hạng quan trọng.

**Kỹ thuật:**
- Log Transformation (log1p): Sử dụng hàm $y = \ln(x + 1)$.
- `log1p` tránh vấn đề log(0) và cung cấp độ chính xác cao hơn cho các giá trị nhỏ

In [13]:
# Log-transform cho train set
number_of_reviews_log_train = np.log1p(number_of_reviews_train)

# Log-transform cho test set
number_of_reviews_log_test = np.log1p(number_of_reviews_test)

print("number_of_reviews transformation:")
print(f"\nTrain set:")
print(f"  - Original range: [{number_of_reviews_train.min():.2f}, {number_of_reviews_train.max():.2f}]")
print(f"  - Log range: [{number_of_reviews_log_train.min():.4f}, {number_of_reviews_log_train.max():.4f}]")
print(f"\nTest set:")
print(f"  - Original range: [{number_of_reviews_test.min():.2f}, {number_of_reviews_test.max():.2f}]")
print(f"  - Log range: [{number_of_reviews_log_test.min():.4f}, {number_of_reviews_log_test.max():.4f}]")

number_of_reviews transformation:

Train set:
  - Original range: [0.00, 629.00]
  - Log range: [0.0000, 6.4457]

Test set:
  - Original range: [0.00, 607.00]
  - Log range: [0.0000, 6.4102]


### 8.5 Tính log-transform của price

**Mục đích:** Giảm độ lệch của phân phối giá và cải thiện hiệu suất mô hình học máy

**Kỹ thuật:**
- Log Transformation (log1p): Sử dụng hàm $y = \ln(x + 1)$.
- `log1p` tránh vấn đề log(0) và cung cấp độ chính xác cao hơn cho các giá trị nhỏ

In [14]:
# Log-transform cho train set
price_log_train = np.log1p(prices_train)

# Log-transform cho test set
price_log_test = np.log1p(prices_test)

print("Feature - Log Price:")
print(f"\nTrain set:")
print(f"  - Original price range: [${prices_train.min():.2f}, ${prices_train.max():.2f}]")
print(f"  - Log price range: [{price_log_train.min():.4f}, {price_log_train.max():.4f}]")
print(f"\nTest set:")
print(f"  - Original price range: [${prices_test.min():.2f}, ${prices_test.max():.2f}]")
print(f"  - Log price range: [{price_log_test.min():.4f}, {price_log_test.max():.4f}]")

Feature - Log Price:

Train set:
  - Original price range: [$10.00, $10000.00]
  - Log price range: [2.3979, 9.2104]

Test set:
  - Original price range: [$10.00, $10000.00]
  - Log price range: [2.3979, 9.2104]


## 9. Target Encoding với Smoothing cho neighbourhood

**Mục đích:** Chuyển đổi biến phân loại `neighbourhood` thành giá trị số dựa trên mối quan hệ với target (price)
- Target encoding giữ được thông tin về mối quan hệ giữa neighbourhood và giá
- Smoothing giúp tránh overfitting cho các neighbourhood có ít samples

**Kỹ thuật:** Target Encoding với Smoothing
- Công thức: `encoded_value = (count * mean + smoothing * global_mean) / (count + smoothing)`
- `smoothing`: hyperparameter điều chỉnh độ tin cậy (thường dùng 10-100)
- Fit trên train set, transform cả train và test để tránh data leakage

In [15]:
def target_encode_with_smoothing(train_cat, train_target, test_cat, smoothing=50):
    """
    Target encoding với smoothing
    
    Parameters:
    - train_cat: categorical values từ train set
    - train_target: target values từ train set (để tính mean)
    - test_cat: categorical values từ test set
    - smoothing: smoothing parameter (default=50)
    
    Returns:
    - train_encoded: target encoded values cho train
    - test_encoded: target encoded values cho test
    """
    # Tính global mean từ train set
    global_mean = np.mean(train_target)
    
    # Lấy unique categories từ train
    unique_cats = np.unique(train_cat)
    
    # Tạo dictionary để lưu encoded values
    encoding_map = {}
    
    for cat in unique_cats:
        # Mask cho category này trong train
        mask = (train_cat == cat)
        
        # Đếm số lượng và tính mean
        count = np.sum(mask)
        cat_mean = np.mean(train_target[mask])
        
        # Áp dụng smoothing
        smoothed_value = (count * cat_mean + smoothing * global_mean) / (count + smoothing)
        encoding_map[cat] = smoothed_value
    
    # Encode train set
    train_encoded = np.array([encoding_map.get(cat, global_mean) for cat in train_cat])
    
    # Encode test set (dùng global_mean cho unseen categories)
    test_encoded = np.array([encoding_map.get(cat, global_mean) for cat in test_cat])
    
    return train_encoded, test_encoded

# Áp dụng target encoding với smoothing cho neighbourhood
te_neighbourhood_train, te_neighbourhood_test = target_encode_with_smoothing(
    neighbourhoods_train, 
    prices_train,  # Sử dụng price làm target
    neighbourhoods_test,
    smoothing=50
)

print("Target Encoding Results:")
print(f"- neighbourhood: {len(np.unique(neighbourhoods_train))} unique values in train")
print(f"\nTrain set:")
print(f"  - Encoded range: [{te_neighbourhood_train.min():.2f}, {te_neighbourhood_train.max():.2f}]")
print(f"  - Mean: {te_neighbourhood_train.mean():.2f}")
print(f"\nTest set:")
print(f"  - Encoded range: [{te_neighbourhood_test.min():.2f}, {te_neighbourhood_test.max():.2f}]")
print(f"  - Mean: {te_neighbourhood_test.mean():.2f}")
print(f"\nMẫu (Original -> Encoded) cho train set:")
for i in range(5):
    print(f"  {neighbourhoods_train[i]} -> {te_neighbourhood_train[i]:.2f}")

Target Encoding Results:
- neighbourhood: 219 unique values in train

Train set:
  - Encoded range: [85.54, 386.43]
  - Mean: 153.55

Test set:
  - Encoded range: [85.54, 386.43]
  - Mean: 154.79

Mẫu (Original -> Encoded) cho train set:
  Chelsea -> 242.83
  East Harlem -> 127.52
  Williamsburg -> 145.00
  Washington Heights -> 95.35
  Harlem -> 120.12
- neighbourhood: 219 unique values in train

Train set:
  - Encoded range: [85.54, 386.43]
  - Mean: 153.55

Test set:
  - Encoded range: [85.54, 386.43]
  - Mean: 154.79

Mẫu (Original -> Encoded) cho train set:
  Chelsea -> 242.83
  East Harlem -> 127.52
  Williamsburg -> 145.00
  Washington Heights -> 95.35
  Harlem -> 120.12


## 10. Chuẩn hóa dữ liệu

**Mục đích:** Chuẩn hóa các features về đoạn [0, 1] để đảm bảo các features có cùng scale

**Kỹ thuật:** Min-Max Scaling
- Formula: `(x - min) / (max - min)`
- **Quan trọng**: Fit trên train set (tính min, max từ train), sau đó transform cả train và test
- Tránh data leakage bằng cách không sử dụng thông tin từ test set

In [16]:
def fit_min_max_scale(train_arr):
    """
    Fit min-max scaler trên train set
    Returns: min_val, max_val, denom
    """
    min_val = np.min(train_arr)
    max_val = np.max(train_arr)
    denom = max_val - min_val
    
    if denom == 0:
        denom = 1  # Tránh chia cho 0
    
    return min_val, max_val, denom

def transform_min_max_scale(arr, min_val, max_val, denom):
    """
    Transform data sử dụng min, max từ train set
    """
    if denom == 0:
        return np.zeros_like(arr)
    return (arr - min_val) / denom

# Danh sách các features cần scale (đã log-transform)
train_features_raw = [
    number_of_reviews_log_train,
    calc_host_listings_log_train,
    availability_train, 
    dist_to_center_train,
    reviews_per_month_train,
]

test_features_raw = [
    number_of_reviews_log_test,
    calc_host_listings_log_test,
    availability_test, 
    dist_to_center_test,
    reviews_per_month_test,
]

feature_names = [
    'number_of_reviews_log',
    'calc_host_listings_log',
    'availability_365',
    'dist_to_center',
    'reviews_per_month',
]

# Fit trên train và transform cả train và test
train_scaled_features = []
test_scaled_features = []
scaling_params = []

for train_f, test_f in zip(train_features_raw, test_features_raw):
    # Fit trên train
    min_val, max_val, denom = fit_min_max_scale(train_f)
    scaling_params.append((min_val, max_val, denom))
    
    # Transform cả train và test
    train_scaled = transform_min_max_scale(train_f, min_val, max_val, denom)
    test_scaled = transform_min_max_scale(test_f, min_val, max_val, denom)
    
    train_scaled_features.append(train_scaled)
    test_scaled_features.append(test_scaled)

print("Min-Max Scaling Results:")
print(f"Đã chuẩn hóa {len(train_scaled_features)} features về đoạn [0, 1]")
print(f"\nKiểm tra range của scaled features:")
print("\nTrain set:")
for name, scaled_f in zip(feature_names, train_scaled_features):
    print(f"  - {name}: [{scaled_f.min():.4f}, {scaled_f.max():.4f}]")

print("\nTest set:")
for name, scaled_f in zip(feature_names, test_scaled_features):
    print(f"  - {name}: [{scaled_f.min():.4f}, {scaled_f.max():.4f}]")
    
print("\nLưu ý: Test set có thể có giá trị ngoài [0,1] nếu có outliers không có trong train")

Min-Max Scaling Results:
Đã chuẩn hóa 5 features về đoạn [0, 1]

Kiểm tra range của scaled features:

Train set:
  - number_of_reviews_log: [0.0000, 1.0000]
  - calc_host_listings_log: [0.0000, 1.0000]
  - availability_365: [0.0000, 1.0000]
  - dist_to_center: [0.0000, 1.0000]
  - reviews_per_month: [0.0000, 1.0000]

Test set:
  - number_of_reviews_log: [0.0000, 0.9945]
  - calc_host_listings_log: [0.0000, 1.0000]
  - availability_365: [0.0000, 1.0000]
  - dist_to_center: [0.0002, 1.0103]
  - reviews_per_month: [0.0000, 0.4778]

Lưu ý: Test set có thể có giá trị ngoài [0,1] nếu có outliers không có trong train


## 11. Array Manipulation - Stacking & Kết hợp features

**Mục đích:** Kết hợp tất cả các features đã xử lý thành ma trận đặc trưng cuối cùng cho train và test

**Kỹ thuật:** Ghép tất cả features thành ma trận cuối cùng
- `np.hstack()`: Ghép ngang (horizontal stacking)
- Reshape 1D arrays thành (N, 1) trước khi stack
- Thứ tự features: [Target Encoded, One-Hot Encoded, Scaled Numerical Features]

In [17]:
# Reshape các mảng 1D thành (N, 1) để có thể stack
train_scaled_reshaped = [f[:, None] for f in train_scaled_features]
test_scaled_reshaped = [f[:, None] for f in test_scaled_features]

# Tạo final matrix cho train set (thêm price_log ở cuối)
final_matrix_train = np.hstack(
    [te_neighbourhood_train[:, None]] +  # Target encoded neighbourhood
    [oh_neighbourhood_group_train] +      # One-hot neighbourhood_group
    [oh_room_type_train] +                # One-hot room_type
    [oh_min_nights_train] +               # One-hot min_nights binned
    train_scaled_reshaped +               # Scaled numerical features
    [price_log_train[:, None]]            # Target variable (price_log)
)

# Tạo final matrix cho test set (thêm price_log ở cuối)
final_matrix_test = np.hstack(
    [te_neighbourhood_test[:, None]] +   # Target encoded neighbourhood
    [oh_neighbourhood_group_test] +       # One-hot neighbourhood_group
    [oh_room_type_test] +                 # One-hot room_type
    [oh_min_nights_test] +                # One-hot min_nights binned
    test_scaled_reshaped +                # Scaled numerical features
    [price_log_test[:, None]]             # Target variable (price_log)
)

print("Final Feature Matrix:")
print(f"\nTrain set:")
print(f"  - Shape: {final_matrix_train.shape}")
print(f"  - Total features: {final_matrix_train.shape[1]}")

print(f"\nTest set:")
print(f"  - Shape: {final_matrix_test.shape}")
print(f"  - Total features: {final_matrix_test.shape[1]}")

print(f"\nBreakdown:")
print(f"  1. Target encoded neighbourhood: 1 feature")
print(f"  2. One-hot room_type: {oh_room_type_train.shape[1]} features")
print(f"  3. One-hot neighbourhood_group: {oh_neighbourhood_group_train.shape[1]} features")
print(f"  4. One-hot min_nights: {oh_min_nights_train.shape[1]} features")
print(f"  5. Scaled numerical features: {len(train_scaled_features)} features")
print(f"  6. Target variable (price_log): 1 feature")
print(f"\nDữ liệu đã sẵn sàng cho Machine Learning!")

Final Feature Matrix:

Train set:
  - Shape: (39107, 19)
  - Total features: 19

Test set:
  - Shape: (9777, 19)
  - Total features: 19

Breakdown:
  1. Target encoded neighbourhood: 1 feature
  2. One-hot room_type: 3 features
  3. One-hot neighbourhood_group: 5 features
  4. One-hot min_nights: 4 features
  5. Scaled numerical features: 5 features
  6. Target variable (price_log): 1 feature

Dữ liệu đã sẵn sàng cho Machine Learning!


## 12. Lưu dữ liệu features dạng CSV

**Mục đích:** Lưu ma trận features đã xử lý dạng CSV riêng cho train và test set
- `train_features.csv`: Dữ liệu train để huấn luyện mô hình
- `test_features.csv`: Dữ liệu test để đánh giá mô hình

In [18]:
# Tạo tên cột cho final_matrix
feature_column_names = ['neighbourhood_target_encoded']

# Thêm tên cột cho one-hot neighbourhood_group
ng_classes = np.unique(neighbourhood_groups_train)
feature_column_names.extend([f'ng_{cls}' for cls in ng_classes])


# Thêm tên cột cho one-hot room_type
room_type_classes = np.unique(room_types_train)
feature_column_names.extend([f'room_{cls}' for cls in room_type_classes])

# Thêm tên cột cho one-hot min_nights
min_nights_labels = ['short_term', 'weekly', 'monthly', 'long_term']
feature_column_names.extend([f'nights_{label}' for label in min_nights_labels])

# Thêm tên các numerical features đã scaled
feature_column_names.extend(feature_names)

# Thêm cột target variable (price_log)
feature_column_names.append('price_log')

# Chuyển final_matrix thành string array để lưu CSV
final_matrix_train_str = final_matrix_train.astype(str)
final_matrix_test_str = final_matrix_test.astype(str)

# Lưu file CSV cho train set
train_csv_path = '../data/processed/train_features.csv'
write_csv(train_csv_path, final_matrix_train_str, np.array(feature_column_names))

# Lưu file CSV cho test set
test_csv_path = '../data/processed/test_features.csv'
write_csv(test_csv_path, final_matrix_test_str, np.array(feature_column_names))

print(f"\n{'='*60}")
print(f"Đã lưu dữ liệu features dạng CSV:")
print(f"\nTrain set:")
print(f"  - File: {train_csv_path}")
print(f"  - Shape: {final_matrix_train.shape}")
print(f"  - Số samples: {final_matrix_train.shape[0]:,}")
print(f"\nTest set:")
print(f"  - File: {test_csv_path}")
print(f"  - Shape: {final_matrix_test.shape}")
print(f"  - Số samples: {final_matrix_test.shape[0]:,}")
print(f"\nSố features: {len(feature_column_names)}")
print(f"\nCác cột trong file CSV:")
for i, col_name in enumerate(feature_column_names):
    if i < len(feature_column_names) - 1:
        print(f"  {i+1}. {col_name}")
    else:
        print(f"  {i+1}. {col_name} ← Target variable")
    
print(f"\n✓ Hoàn tất! Dữ liệu đã được chia và lưu riêng để tránh data leakage.")
print(f"✓ Cột price_log (target variable) đã được thêm vào cuối cùng.")

Đã lưu dữ liệu vào: ../data/processed/train_features.csv
Shape: (39107, 19)
Số cột: 19
Đã lưu dữ liệu vào: ../data/processed/test_features.csv
Shape: (9777, 19)
Số cột: 19

Đã lưu dữ liệu features dạng CSV:

Train set:
  - File: ../data/processed/train_features.csv
  - Shape: (39107, 19)
  - Số samples: 39,107

Test set:
  - File: ../data/processed/test_features.csv
  - Shape: (9777, 19)
  - Số samples: 9,777

Số features: 19

Các cột trong file CSV:
  1. neighbourhood_target_encoded
  2. ng_Bronx
  3. ng_Brooklyn
  4. ng_Manhattan
  5. ng_Queens
  6. ng_Staten Island
  7. room_Entire home/apt
  8. room_Private room
  9. room_Shared room
  10. nights_short_term
  11. nights_weekly
  12. nights_monthly
  13. nights_long_term
  14. number_of_reviews_log
  15. calc_host_listings_log
  16. availability_365
  17. dist_to_center
  18. reviews_per_month
  19. price_log ← Target variable

✓ Hoàn tất! Dữ liệu đã được chia và lưu riêng để tránh data leakage.
✓ Cột price_log (target variable) đã 